# Software para generación de anotaciones en formato YOLOv4
* **Autor:** Julian Zuloaga
* **Asignatura:** Proyecto Electrónico
* **Fecha:** 03 de julio de 2022
---

**Descripción:** Este cuaderno de Google Colab presenta la implementación de un software para convertir bases de datos para FER al formato YOLOv4. En este caso, la demostración se realiza con un extracto de la base de datos AffectNet, sin embargo, adaptar el código de este programa para ser utilizado con otras bases de datos es muy fácil.

* **Nota: Se recomienda hacer una copia de este cuaderno en su unidad de Google Drive personal antes de ejecutar las celdas de código.**
---

## 1) Se importan recursos
El primer paso es cargar los archivos y recursos necesarios para el funcionamiento del programa.

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
%matplotlib inline
# Se importan librerías para procesamiento de imágenes
from PIL import Image
import ntpath
import os
import glob

In [ ]:
# Se clona el repositorio con Darknet
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Se modifica el archivo makefile para habilitar GPU, OPENCV y LIBSO
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

/content/darknet


In [ ]:
# Se construye Darknet para contar con el archivo darknet.py junto con sus dependencias
!make

In [ ]:
# Se descargan desde GDrive los pesos del modelo Yolov4 para Detección Facial que se entrenó en la asignatura Proyecto Electrónico
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1emhZyDxGmHWp2UuIpYdi59UuoJxYRc5B' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1emhZyDxGmHWp2UuIpYdi59UuoJxYRc5B" -O yolov4-csp.weights && rm -rf /tmp/cookies.txt

In [ ]:
# Se crea el directorio custom_model en la carpeta darkent
!mkdir /darknet/ custom_model

In [ ]:
# Luego se descarga el archivo de configuración yolov4-csp.cfg
!wget "https://drive.google.com/uc?export=download&id=1qTLjZIVRMLklWAcZhdF09x_pLvTWxw1y" -O yolov4-csp.cfg
# Se mueve el archivo de configuración a la carpeta "custom_model"
!cp yolov4-csp.cfg ../darknet/custom_model/

In [ ]:
# Después, se descargan los archivos obj.names y obj.data, y se copian en sus respectivas carpetas
!wget "https://drive.google.com/uc?export=download&id=1OkzofUrQInBNKszZSwa88EZr2rojYqLr" -O obj.names
!wget "https://drive.google.com/uc?export=download&id=1R1XdN8QyaneplyWTAkD1DojV9_jCyzUD" -O obj.data
# Se mueve el archivo de configuración a la carpeta "custom_model"
!cp obj.names ../darknet/custom_model/
!cp obj.data ../darknet/custom_model/
# Adicionalmente, se copia el archivo .names a la carpeta data
!cp obj.names ../darknet/data/

In [ ]:
# Se copian los archivos de configuración a la carpeta cfg
!cp obj.data ../darknet/cfg/
!cp obj.names ../darknet/cfg/
!cp yolov4-csp.cfg ../darknet/cfg/

## 2) Se definen funciones auxiliares de Darknet para Python
Para utilizar YOLOv4 con Python, se usará algunas de las funciones integradas que posee este framework, las cuales están contenidas en el archivo darknet.py. Para ello, se importarán estas funciones al espacio de trabajo.

In [ ]:
# Se importan las funciones de Darknet
from darknet import *
# Se carga nuestra red perzonalizada de YOLOv4 para Detección de Rostros
network, class_names, class_colors = load_network("cfg/yolov4-csp.cfg", "cfg/obj.data", "yolov4-csp.weights")

# Se calculan las dimensiones de entrada de la red neuronal
width = network_width(network)
height = network_height(network)

# Se define función auxiliar para ejecutar detecciones con la red neuronal
def darknet_helper(img, width, height):
  darknet_image = make_image(width, height, 3) # imagen en formato Darknet
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # se pasa imagen de BGR a RGB
  img_resized = cv2.resize(img_rgb, (width, height), # Se escala imagen a dimensiones de entrada de la red
                              interpolation=cv2.INTER_LINEAR)
  # Se obtienen las relaciones para convertir bounding boxes al tamaño real de la imagen
  img_height, img_width, _ = img.shape # tamaño imagen
  width_ratio = img_width/width # razón de ancho = ancho imagen/ancho entrada red
  height_ratio = img_height/height

  # Se ejecutan las detecciones con la red sobre la imagen entregada
  copy_image_from_bytes(darknet_image, img_resized.tobytes())
  detections = detect_image(network, class_names, darknet_image)
  free_image(darknet_image)
  return detections, width_ratio, height_ratio

## 3) Prueba de detección facial con YOLOv4
Se ejecuta una detección de prueba para verificar que nuestra red de YOLOv4 para detectar rostros en imágenes ha sido cargada correctamente.


In [ ]:
# Se descarga imagen de prueba
!wget "https://drive.google.com/uc?export=download&id=1YIF--yDHT3QCvCxOxf12mpBbDpy5S3ri" -O portrait.jpg
!cp portrait.jpg ../darknet/data/

In [ ]:
# Se ejecuta la prueba sobre la imagen portrait.jpg
image = cv2.imread("data/portrait.jpg") # se abre la imagen con cv2
detections, width_ratio, height_ratio = darknet_helper(image, width, height) # Se ejecuta la detección sobre la imagen
# Se generan bounding boxes sobre las detecciones
for label, confidence, bbox in detections:
  left, top, right, bottom = bbox2points(bbox) #coordenadas del bounding box
  # se convierte cooredenadas en formato YOLO a cordenadas de la imagen cargada
  left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
  # se genera rectángulo sobre las coordenadas obtenidas
  cv2.rectangle(image, (left, top), (right, bottom), class_colors[label], 2)
  # Se escribe clase junto con probabilidad de confianza
  cv2.putText(image, "{} [{:.2f}]".format(label, float(confidence)),
                    (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.2,
                    class_colors[label], 2)
cv2_imshow(image) # se muestra la imagen

## 4) Se descarga la base de datos AffectNet
Se descarga el comprimido con el extracto de la base de datos AffectNet que se convertirá a formato YOLOv4.

In [ ]:
# Se descarga la base de datos AffectNet
%cd /content/
!mkdir "dataset" # se crea carpeta "dataset"
# EN ESTE PASO, CARGUE SU VERSIÓN DE AFFECTNET EN "/content/dataset/AffectNet_JPG_formated.zip"
# Se descomprime la base de datos
%cd /content/dataset/
!unzip /content/dataset/AffectNet_JPG_formated.zip
!rm /content/dataset/AffectNet_JPG_formated.zip #se borra el comprimido
%cd /content/

## 5) Configuración del procesador de imágenes

Ahora se establecen algunos parámetros importantes para el funcionamiento del programa que genera las anotaciones de la base de datos cargada, como los directorios de entrada y salida.

In [ ]:
# Se crea directorio de destino
!mkdir "/content/YOLOv4_formatted_dataset"

In [ ]:
# Se define dirección de entrada y salida del sistema de procesamiento
data_dir = "/content/dataset/" # entrada
output_dir = "/content/YOLOv4_formatted_dataset/" # salida
# Se extraen las subdirecciones de las carpetas con las clases
zz = [x[0] for x in os.walk(data_dir)]
print(f"Se han encontrado {len(zz)-1} clases:")
# Se extraen los nombres de las clases
lista_clases = list() #Se inicializa lista con nombre de clases
# Se imprimen las clases encontradas
for j in range(1,len(zz)):
	lista_clases.append(ntpath.basename(zz[j]))
	print(f'{ntpath.basename(zz[j])}')
 
#se define lista con las clases de la base de datos en orden alfabético
classes_names = ["anger","contempt","disgust","fear","happy","neutral","sad","surprise"]


In [ ]:
# Se crean subdirectorios obj y test en carpeta de destino
os.mkdir(output_dir+"/test")
os.mkdir(output_dir+"/obj")

## 6) Se ejecuta la conversión de formatos
Luego, se procede a ejecutar la conversión.

In [ ]:
# Se comienza con la conversión de las imágenes
# se definen las clases en el orden que acepta YOLOv4 para FER (obj.names)
img_dir = list() # se inicializa lista con directorio de imágenes
img_path = "" #se inicializa string con directorio de imagen
contador_errores = 0 #se cuentan las imágenes no convertidas
lista_errores = list() #lista para llevar seguimiento de los errores de conversión
porcen_train = 0.85 #se define % de imágenes del dataset para entrenamiento
porcen_test = 1-porcen_train # se define % de imágenes para validación
img_name_counter = 1 # contador de imágenes para asignar nombre
# Se itera la lista de clases
for j in lista_clases:
    # Se llena la lista con las direcciones de las imágenes de una clase
    img_dir = glob.glob(data_dir+j+"/*.jpg")
    # e calculan imágenes para entrenamiento y validación
    n_img_train = int(len(img_dir)*porcen_train)
    n_img_test = int(len(img_dir)-n_img_train)
    print(f"Total de imágenes en '{j}' es: {len(img_dir)}. Imágenes para entrenamiento: {n_img_train}, validación: {n_img_test}.")
    img_counter = 1 # contador de imágenes procesadas por clase para división de dataset
    # Se itera la lista de imágenes de una clase
    for i in img_dir:
      # Se abre la imagen con Pillow
      img = Image.open(i)
      # Se obtiene las dimensiones de la imagen
      img_width, img_height = img.size #(width, height)
      # se obtiene el nombre de la imagen sin la extensión
      #img_name = (os.path.split(i[:-4])[-1])
      # Se asigna nombre a la imagen con el número y clase
      img_name = img_name_counter
      # Se lee la imagen con cv2
      img_cv2 = cv2.imread(i) # se abre la imagen con cv2
      # Se ejecuta detección
      detections, width_ratio, height_ratio = darknet_helper(img_cv2, width, height) # Se ejecuta la detección sobre la imagen
      # Se generan bounding boxes sobre las detecciones
      for label, confidence, bbox in detections:
        left, top, right, bottom = bbox2points(bbox) #coordenadas del bounding box
        # se convierte cooredenadas en formato YOLO a cordenadas de la imagen cargada
        left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
      # Se calculan parámetros para archivo de texto:
      param1 = classes_names.index(j) # clase
      param2 = (((right - left)/2) + left)*(1/img_width) # Centro horizontal normalizado
      param3 = ((bottom - top)/2 + top)*(1/img_height) # Centro vertical normalizado
      param4 = (right - left)/img_width # ancho bbox normalizado
      param5 = (bottom - top)/img_height # alto bbox normalizado
      # Se guardan las coordenadas en un archivo de texto
      if img_counter <= n_img_train:
        img_txt = open(output_dir + "obj/" + str(img_name) + ".txt",'w')
        cv2.imwrite(output_dir + "obj/" + str(img_name) + ".jpg", img_cv2)
      else:
        img_txt = open(output_dir + "test/" + str(img_name) + ".txt",'w')
        cv2.imwrite(output_dir + "test/" + str(img_name) + ".jpg", img_cv2)
      # Se escriben los parámetros en el archivo de text
      img_txt.write(f'{param1} {param2} {param3} {param4} {param5}')
      # Se cierra el archivo de texto
      img_txt.close()
      #Se cierra la imagen abierta con pillow
      img.close()
      #Se incrementan los contadores de imágenes procesadas
      img_counter = img_counter + 1
      img_name_counter = img_name_counter + 1
      
print("Conversión Finalizada!!")

## 7) Guardado de la base de datos convertida
Posterior a la conversión, se procede a guardar la base de datos generada.

In [ ]:
# Se comprime el dataset obtenido
!zip -r /content/YOLOv4_formatted_dataset.zip /content/YOLOv4_formatted_dataset

Aquí hay 2 opciones para guardar la base de datos convertida, la primera opción es descargarla directamente a la computador del Host, desde la Cloud VM de Google Colab, aunque esta opción es bastante lenta. La segunda opción y la más recomendada, es montar una unidad de Google Drive y copiar el archivo directamente a la unidad, lo cual es mucho más rápido.

In [ ]:
# Se define función auxiliar para descargar modelo de pesos desde la Cloud VM al computador local
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
# Se descarga el dataset guardado al computador local
download("/content/YOLOv4_formatted_dataset.zip")

# Nota: El .zip contiene el dataset en la estructura: content > cropped_dataset > angry, contempt, disgust, etc.

In [ ]:
# Se monta unidad de GDrive para copiar más rápido el Dataset comprimido
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Se crea un symbolic link para simplificar escritura de directorio
!ln -s /content/drive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# Se copia el dataset al GDrive
!cp /content/YOLOv4_formatted_dataset.zip /content/drive/MyDrive

Con eso concluye la implementación del software. Muchas Gracias.